# Read and process a cross section file in *ALEPH* format

In [1]:
import os
from tempfile import TemporaryDirectory
import numpy as np

In [2]:
%%capture --no-stdout
import sandy
from sandy import aleph2 as al2

This is an example of what one can find in a ALEPH-2 cross section file tabulated in the so-called *aleph* format.

In [3]:
string = """              952410                   4                   4  200.00000000000000
  5.000000000000E+00
  1.000000000000E-11  1.000000000000E-05  1.000000000000E-01  1.000000000000E+01
              952420                   0  5.000000000000E+00
  1.0000000000000000  2.0000000000000000  3.0000000000000000  4.0000000000000000
                  18                   0  2.000000000000E+02
  1.0000000000000000  2.0000000000000000  3.0000000000000000  4.0000000000000000
              942410                   2  5.000000000000E+00
  6.0000000000000000  5.0000000000000000
               10010                   2  7.000000000000E-01
  6.0000000000000000  5.0000000000000000
"""

In [4]:
xsfile = al2.AlephFile.from_text(string)
xsfile

ALEPH file for nuclide: 952410

In [5]:
xsfile.data

{'nuclide': 952410,
 'energies': array([1.e-11, 1.e-05, 1.e-01, 1.e+01]),
 'fission_qvalue': 200.0,
 'capture_qvalue': 5.0,
 'reactions': {952420: {'qval': 5.0, 'xs': array([1., 2., 3., 4.])},
  18: {'qval': 200.0, 'xs': array([1., 2., 3., 4.])},
  942410: {'qval': 5.0, 'xs': array([6., 5.])},
  10010: {'qval': 0.7, 'xs': array([6., 5.])},
  4581: {'qval': 0.0, 'xs': array([200., 400., 600., 800.])},
  4582: {'qval': 0.0, 'xs': array([ 5., 10., 15., 20.])}}}

Add a fake threshold reaction. 

In [6]:
ID = 100
new = [3, 4, 5]
xsfile_upd = xsfile.add_reaction(ID, new)

In [7]:
print(xsfile_upd.to_string())

              952410                   5                   4  2.000000000000E+02
  5.000000000000E+00                                                            
  1.000000000000E-11  1.000000000000E-05  1.000000000000E-01  1.000000000000E+01
                  18                   0  2.000000000000E+02                    
  1.000000000000E+00  2.000000000000E+00  3.000000000000E+00  4.000000000000E+00
                 100                   1  0.000000000000E+00                    
  3.000000000000E+00  4.000000000000E+00  5.000000000000E+00                    
               10010                   2  7.000000000000E-01                    
  6.000000000000E+00  5.000000000000E+00                                        
              942410                   2  5.000000000000E+00                    
  6.000000000000E+00  5.000000000000E+00                                        
              952420                   0  5.000000000000E+00                    
  1.000000000000E+00  2.0000

## From *aleph* format to HDF5

Save cross section data into HDF5 file.

In [8]:
filename = "test.h5"
library = "test_library"
temperature = 652.666

path = os.getcwd()
with TemporaryDirectory(dir=path) as tmpdirname:
    h5file = os.path.join(tmpdirname, filename)
    xsfile.to_hdf5(h5file, library=library, temperature=temperature)

Adding 'test_library/xs/652.7/952410' to file'C:\Users\lfiorito\sandy_notebooks\tmpjou9qnd2\test.h5'...


Cross section data are appended to a HDF5 file in a ['fixed' format](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_hdf.html) and in a namespace defined by four keys:
 - library name: all lowercase, regex `(?P<version>[a-z]+)_(?P<version>[0-9]+)`;
 - data type: key `xs` always used for cross section data in `aleph` format;
 - temperature: temperature in kelvin written as `{:f.1}`;
 - ZAM: nuclide identifier of charge, mass and isomeric number.

## From aleph format to a `pandas` dataframe.

In [9]:
df = xsfile.to_dataframe()
df

PARENT           952410                                                  \
DAUGHTER         952420      18          942410      10010       4581     
E                                                                         
1.00000e-11 1.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 2.00000e+02   
1.00000e-05 2.00000e+00 2.00000e+00 0.00000e+00 0.00000e+00 4.00000e+02   
1.00000e-01 3.00000e+00 3.00000e+00 6.00000e+00 6.00000e+00 6.00000e+02   
1.00000e+01 4.00000e+00 4.00000e+00 5.00000e+00 5.00000e+00 8.00000e+02   

PARENT                   
DAUGHTER         4582    
E                        
1.00000e-11 5.00000e+00  
1.00000e-05 1.00000e+01  
1.00000e-01 1.50000e+01  
1.00000e+01 2.00000e+01

In [10]:
xs = sandy.Xs(df)
xs

MAT              952410                                                  \
MT               952420      18          942410      10010       4581     
E                                                                         
1.00000e-11 1.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 2.00000e+02   
1.00000e-05 2.00000e+00 2.00000e+00 0.00000e+00 0.00000e+00 4.00000e+02   
1.00000e-01 3.00000e+00 3.00000e+00 6.00000e+00 6.00000e+00 6.00000e+02   
1.00000e+01 4.00000e+00 4.00000e+00 5.00000e+00 5.00000e+00 8.00000e+02   

MAT                      
MT               4582    
E                        
1.00000e-11 5.00000e+00  
1.00000e-05 1.00000e+01  
1.00000e-01 1.50000e+01  
1.00000e+01 2.00000e+01  